In [2]:
# imports
import pandas as pd
import numpy as np

import os
from os import listdir
from os.path import isfile, join

import keras
import joblib

from sklearn.svm import SVC
import cv2
from matplotlib import pyplot as plt
print "loaded imports"

Using TensorFlow backend.


loaded imports


In [ ]:
#load images
folders = [d for d in listdir("./train/") if not isfile(join("./train/", d)) and d not in ["cropped"]]

training_data = np.array([])
training_labels = np.array([])

for category in folders:
    print category
    images = [file for file in listdir("./train/"+category)]
    np.random.shuffle(images)
    images = images[0:300] #limit number of images used
    
    for image in images:
        hog = cv2.HOGDescriptor()
        img = cv2.imread("./train/" + category + "/" + image, 0)
        img = cv2.resize(img, (400, 250))
        h = hog.compute(img)
        h = h.astype(np.float64)
        np.random.shuffle(h)
        h = h[0:100,:] # trim vector so all are same size
        vector_data = h.reshape(1,100) 
        
        if len(training_data) == 0:
            training_data = np.append(training_data, vector_data)
            training_data = training_data.reshape(1,100)
        else:
            training_data   = np.concatenate((training_data, vector_data), axis=0)
        training_labels = np.append(training_labels,category)
        
print "finished"

ALB
BET
DOL
LAG
NoF
OTHER
SHARK


In [24]:
X = training_data
y = training_labels
y = y.reshape(y.shape[0],)

# Create and fit the SVM
# Fitting should take a few minutes
clf = SVC(kernel='linear', C = 1.0, probability=True)
clf.fit(X,y)
joblib.dump(clf, 'fishy_svm.pkl')

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [7]:
# clf = joblib.load('fishy_svm.pkl') # uncomment to use generated
fishPredict = cv2.imread('./train/LAG/img_00657.jpg', 0)  # Correct is LAG --> Class 3
hog = cv2.HOGDescriptor()
fishPredict = cv2.resize(fishPredict, (400, 250))
h = hog.compute(fishPredict)
h = h.astype(np.float64)
np.random.shuffle(h)
h = h[0:100,:] # trim vector so all are same size
vector_data = h.reshape(1,100)

print("Linear SVM Prediction:")
print(clf.predict(vector_data))        # prints highest probability class, only
print(clf.predict_proba(vector_data))

joblib.dump(clf, 'fishy_svm.pkl')
    # to load SVM model, use:  clf = joblib.load('filename.pkl')

Linear SVM Prediction:
['SHARK']
[[ 0.1382508   0.15924064  0.14877601  0.09481978  0.13919176  0.16176548
   0.15795553]]


['fishy_svm.pkl']

In [ ]:
clf = joblib.load('fishy_svm.pkl')
hog = cv2.HOGDescriptor()
def getPrediction(filename):
    fish_image = cv2.imread(filename, 0)
    fish_image = cv2.resize(fish_image, (400, 250))
    fish_h = hog.compute(fish_image)
    fish_h = fish_h.astype(np.float64)
    np.random.shuffle(fish_h)
    fish_h = fish_h[0:100,:]
    vector_data = fish_h.reshape(1,100)
    return clf.predict_proba(vector_data)

def savePredictions():
    headers = ["image", "ALB", "BET", "DOL", "LAG", "NoF", "OTHER", "SHARK", "YFT"]
    predictions = np.array([])
    maxi = 0;
    for file in listdir("./test_stg1/"):
        if maxi > 0:
            continue
        else:
            maxi +=1
        prediction = getPrediction("./test_stg1/"+file)
        prediction = np.insert(prediction, 0, 0)
        prediction_s = np.array(prediction, dtype='|S4')
        print prediction_s
        prediction_s = np.insert(prediction_s, 0, file)
        print prediction_s
        np.append(predictions, prediction_s)
        continue
    
    df = pd.DataFrame(predictions, columns=headers)
    df.to_csv('test.csv', index=False, header=True, sep=',')
    
savePredictions()
        